In [1]:
! pip uninstall pulse2percept -y
! cd pulse2percept && pip install -r requirements.txt && pip install -r requirements-dev.txt && pip install -e .
! pip install tqdm 

Uninstalling pulse2percept-0.6.0.dev0:
  Successfully uninstalled pulse2percept-0.6.0.dev0


Obtaining file:///home/jovyan/work/pulse2percept
  Running setup.py develop for pulse2percept


In [2]:
import pulse2percept as p2p
from prima import * 
%matplotlib inline 
import matplotlib.pyplot as plt 

In [3]:
model = p2p.models.AxonMapModel(rho=100, axlambda=200)
model.engine = 'serial'
model.build()

AxonMapModel(ax_segments_range=(3, 50), axlambda=200, 
             axon_pickle='axons.pickle', 
             axons_range=(-180, 180), engine='serial', 
             eye='RE', grid_type='rectangular', 
             ignore_pickle=False, loc_od_x=15.5, 
             loc_od_y=1.5, n_ax_segments=500, n_axons=500, 
             n_jobs=-1, rho=100, scheduler='threading', 
             thresh_percept=0, verbose=True, 
             xrange=(-20, 20), xystep=0.25, 
             yrange=(-15, 15))

In [5]:
from tqdm import tqdm 
from multiprocessing import Pool

def generate_dataset_helper(argpack):
    model, implant, stim_dim, idx = argpack
    stim = np.zeros(stim_dim)
    stim[idx] = 1
    implant.stim = stim
    print("generating percepts for index: " + str(idx))
    percept = model.predict_percept(implant)
    print("done generating percept for index: " + str(idx))
    return percept

def generate_dataset(model, implant, stim_dim, j=8):
    def helper(idx):
        stim = np.zeros(stim_dim)
        stim[idx] = 1
        implant.stim = stim
        return model.predict_percept(implant)
    return [helper(idx) for idx in tqdm(range(0, stim_dim))]

def generate_dataset_fast(model, implant, stim_dim, j=8):
    with Pool(j) as p:
        return list(p.map(generate_dataset_helper, [(model, implant, stim_dim, idx) for idx in tqdm(range(0, stim_dim))]))

In [6]:
implant = p2p.implants.ArgusII()
output_percepts = generate_dataset_fast(model, implant, 60)

100%|██████████| 60/60 [00:00<00:00, 36188.99it/s]


generating percepts for index: 0
generating percepts for index: 2
generating percepts for index: 4
generating percepts for index: 6
generating percepts for index: 8
generating percepts for index: 10
generating percepts for index: 12
generating percepts for index: 14
done generating percept for index: 0
generating percepts for index: 1
done generating percept for index: 2
generating percepts for index: 3
done generating percept for index: 4
generating percepts for index: 5
done generating percept for index: 6
generating percepts for index: 7
done generating percept for index: 8
generating percepts for index: 9
done generating percept for index: 10
generating percepts for index: 11
done generating percept for index: 12
generating percepts for index: 13
done generating percept for index: 14
generating percepts for index: 15
done generating percept for index: 1
generating percepts for index: 16
done generating percept for index: 3
done generating percept for index: 5
generating percepts fo

In [10]:
import pickle
pickle.dump(output_percepts, open("./data/ArgusII_percepts.pickle", "wb"))